## Model/data owner
The data owner will train and save a model (this notebook) and ensure the model reproductivity (Model_Reproduction.ipynb)


## Get the dataset

In [1]:
# Setup
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

In [2]:
# define dataset path
dataset_path = '../Datasets/CIFAR10'
img_path = '../Datasets/CIFAR10/images'

In [ ]:
# Get the data file and all of the labels
import pandas as pd
data_df = pd.read_csv(f'{dataset_path}/data.csv')
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
label2id = {i:k for i, k in enumerate(classes)}
print('\n'.join([f'{k}\t{i}' for i, k in enumerate(classes)]))

plane	0
car	1
bird	2
cat	3
deer	4
dog	5
frog	6
horse	7
ship	8
truck	9


In [ ]:
# get training device
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# User A has 2000 images of frog(6), 1000 images of bird(2), and 1000 images of others
data = data_df.copy()
data_A_idx = data.query('label == 6').sample(2000).index.tolist() + \
             data.query('label == 2').sample(1000).index.tolist() + \
             data.query('label not in (2,6)').sample(1000).index.tolist()
data_A = data.iloc[data_A_idx].reset_index(drop=True)
data_df__A = data.iloc[[i for i in data.index if i not in data_A_idx]].reset_index(drop=True) # Get the data without A

A_label_map = {
    0: 'others',
    1: 'frog',
    2: 'bird'
}
A_label2id = {v:k for k,v in A_label_map.items()}
data_A.label = data_A.label_name.apply(lambda x: A_label2id[x] if x in A_label2id else 0)

In [ ]:
# In future, the above process of selecting dataset would use the get_dataset function from Dataset_Splitting.ipynb

In [ ]:
# User B has 3000 images of frog(6), 2000 images of ship(8), 1000 images of horses(7), and 1000 others
# data = data_df__A.copy()
# data_B_idx = data.query('label == 6').sample(3000).index.tolist() + \
#              data.query('label == 8').sample(2000).index.tolist() + \
#              data.query('label == 7').sample(1000).index.tolist() + \
#              data.query('label not in (6,7,8)').sample(1000).index.tolist()
# data_B = data.iloc[data_B_idx].reset_index(drop=True)
# data_df__AB = data.iloc[[i for i in data.index if i not in data_B_idx]].reset_index(drop=True) # Get the data without A and B

# B_label_map = {
#     0: 'others',
#     1: 'frog',
#     2: 'ship',
#     3: 'horse',
# }
# B_label2id = {v:k for k,v in B_label_map.items()}
# data_B.label = data_B.label_name.apply(lambda x: B_label2id[x] if x in B_label2id else 0)

In [ ]:
# User C has 1000 images of frog(6), 1000 images of ship(8), 1000 images of horses(7), 2000 images of planes(0) and 1000 others
# data = data_df__AB.copy()
# data_C_idx = data.query('label == 6').sample(1000).index.tolist() + \
#              data.query('label == 8').sample(1000).index.tolist() + \
#              data.query('label == 7').sample(1000).index.tolist() + \
#              data.query('label == 0').sample(2000).index.tolist() + \
#              data.query('label not in (0,6,7,8)').sample(1000).index.tolist()
# data_C = data.iloc[data_C_idx].reset_index(drop=True)
# data_df__ABC = data.iloc[[i for i in data.index if i not in data_C_idx]].reset_index(drop=True) # Get the data without A, B and C

# C_label_map = {
#     0: 'others',
#     1: 'frog',
#     2: 'ship',
#     3: 'horse',
# }
# C_label2id = {v:k for k,v in C_label_map.items()}
# data_C.label = data_C.label_name.apply(lambda x: C_label2id[x] if x in C_label2id else 0)

In [ ]:
# download vision transformer model 
from transformers import ViTFeatureExtractor, ViTForImageClassification
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
vit_model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', output_hidden_states=True).to(device)

# freeze the pre-trained model
vit_model.eval()
for param in vit_model.parameters():
    param.requires_grad = False

In [ ]:
import torch
from torch import nn
import numpy as np
from PIL import Image

import albumentations as A

# create custom dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data, feature_extractor, vit_model, transform):
        self.images = [f'{img_path}/{i}' for i in data.image]
        self.labels = data.label.tolist()
        self.transform = transform
        self.feature_extractor = feature_extractor

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = Image.open(self.images[index])
        if self.transform is not None:
            image = np.array(image)
            image = Image.fromarray(self.transform(image=image)['image'], 'RGB')
        image = self.feature_extractor(image, return_tensors='pt')['pixel_values'][0]
        logits = vit_model(image.unsqueeze(0).to(device)).hidden_states[-1][:,-1,:].cpu().squeeze().detach()
        return logits, self.labels[index]

def get_loader(data, feature_extractor, vit_model, transform, batch_size=32, shuffle=True):
    return torch.utils.data.DataLoader(Dataset(data, feature_extractor, vit_model, transform), batch_size=batch_size, shuffle=shuffle)

transform = A.Compose([
    A.Resize(256, 256),
    A.RandomCrop(224, 224),
    A.Flip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5)
])

A_loader = get_loader(data_A, feature_extractor, vit_model, transform)
# B_loader = get_loader(data_B, feature_extractor, vit_model, transform)
# C_loader = get_loader(data_C, feature_extractor, vit_model, transform)

In [ ]:
# initialize model, criterion and optimizer
fc_net = nn.Sequential(
    nn.LazyBatchNorm1d(),
    nn.LazyLinear(128),
    nn.GELU(),
    nn.LazyLinear(len(A_label_map))
).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fc_net.parameters(), lr=0.001)

In [ ]:
# A simple training process for 5 epochs
from tqdm import tqdm
fc_net.train()
for i in range(5):
    for batch in tqdm(A_loader):
        logits = fc_net(batch[0].to(device))
        true_labels = batch[1].long().to(device)
        optimizer.zero_grad()
        loss = criterion(logits, true_labels)
        loss.backward()
        optimizer.step()


100%|██████████| 125/125 [01:21<00:00,  1.54it/s]


In [ ]:
# TODO: model evaluation
sample_batch_A = next(iter(A_loader))
fc_net.eval()
with torch.no_grad():
    logits = fc_net(sample_batch_A[0].to(device))
    y_pred = logits.argmax(1).detach().cpu().numpy()
    y_true = sample_batch_A[0].argmax(1)
    print('Batch accuracy:', np.mean(y_pred == y_true))


In [ ]:
# save the output model as .pt file
torch.save(fc_net, f'{dataset_path}/saved_models/A_model_1.pt')